# 0. 실행 전 필수 안내

* 본 코드는 GPU 환경에서 실행하는 것을 강력히 권장합니다!

     
     Colab 메뉴: [런타임] → [런타임 유형 변경] → 'GPU' 선택
     
      → 이미지 처리, 모델 학습, 추론 속도가 크게 개선됩니다.

### 주요 라이브러리:
- transformers: 사전학습된 모델 불러오기 및 미세조정 (🤗 Hugging Face)
- timm: Vision Transformer 계열 모델 사용 시 필요한 라이브러리
- scikit-learn: 성능 평가 지표 계산 (정확도, 정밀도, 재현율, F1 등)
- tqdm: 학습 루프 진행률 시각화
- PIL (Pillow): 이미지 불러오기 및 전처리

# 1. 환경 설정 및 기본 설치

In [ ]:
# 필수 라이브러리를 설치합니다.
# Colab 환경에서는 최초 실행 시 한 번만 설치하면 됩니다.

!pip install transformers timm scikit-learn

# Google Drive를 마운트하여 데이터를 불러올 수 있도록 설정합니다.
# 코드 실행 시 나타나는 인증 절차를 완료해주세요.
from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [ ]:
import zipfile
import io
import torch
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from transformers import AutoImageProcessor, AutoModelForImageClassification
from torch.optim import AdamW
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# 2. 경로 및 데이터 로딩
* 본 예제에서는 데이터가 압축(zip) 형태로 Drive에 저장되어 있다고 가정합니다.
* 본인의 Drive에 저장된 zip 파일 경로를 확인해 아래 경로들을 수정해주세요.
* 웹크롤링을 통한 Inference 또한 가능합니다.

In [ ]:
# XX Model로 생성한 fake 이미지 (N장 - 학습용)
xx_fake_path = "/content/drive/MyDrive/Colab Notebooks/Deepfake_Detection/XX_fake_N.zip"

# 평가용 real 이미지 (예: XX 인물 이미지)
xx_real_path = "/content/drive/MyDrive/Colab Notebooks/Deepfake_Detection/XX_real.zip"

# 평가용 fake 이미지 (예: XX으로 생성한 50장)
xx_fake_path2 = "/content/drive/MyDrive/Colab Notebooks/Deepfake_Detection/XX_fake.zip"

# 3. 데이터 로딩 및 함수 정의

In [ ]:
# Fake 이미지 로딩 함수 (학습용)
def load_fake_images(zip_path):
    """
    zip 파일 내 이미지들을 불러와 (이미지이름, 이미지바이트, 라벨) 튜플 리스트로 반환합니다.
    라벨 1: FAKE
    """
    fake_data = []
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        for name in zip_ref.namelist():
            if name.endswith(('.jpg', '.png')):
                fake_data.append((name, zip_ref.read(name), 1))
    return fake_data

# Custom Dataset 클래스 정의
class FakeDataset(Dataset):
    def __init__(self, data, processor):
        self.data = data
        self.processor = processor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        name, img_bytes, label = self.data[idx]
        image = Image.open(io.BytesIO(img_bytes)).convert("RGB")
        inputs = self.processor(images=image, return_tensors="pt")
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        inputs["labels"] = torch.tensor(label, dtype=torch.long)
        return inputs

# Real 이미지 로딩 함수 (평가용)
def load_real_images(zip_path, limit=50):
    """
    zip 내 최대 50장의 real 이미지를 불러옵니다.
    """
    real_images = []
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        for name in zip_ref.namelist():
            if name.endswith(('.jpg', '.png')):
                real_images.append((name, zip_ref.read(name)))
                if len(real_images) >= limit:
                    break
    return real_images

# Real 이미지 불러오기
xx_real = load_real_images(xx_real_path)

# 4. 사전 학습된 모델 평가 (Inference)
평가 함수 정의

→ fake와 real 이미지를 넣어 모델의 성능을 평가합니다.

→ Softmax 확률 기반으로 best threshold를 자동 탐색합니다.

### "FAKE 기준"이란?

→ 성능 지표 계산 시 **FAKE 클래스(=1로 레이블링된 클래스)**를 기준으로 삼았다는 의미입니다.

→ 예를 들어, precision은 "FAKE라고 예측한 것 중 진짜 FAKE 비율",

    recall은 "실제 FAKE 중에서 맞게 예측한 비율"을 의미합니다.

### Threshold 탐색 이유?

→ 모델 출력은 softmax 확률이므로, 정확한 예측을 위해 확률값을 기준으로
   
    예측 기준선을 (0.05 ~ 0.99 사이에서) 최적화합니다.

In [ ]:
# 사전 학습 모델 로딩
processor = AutoImageProcessor.from_pretrained("Pretrained_Model")
model = AutoModelForImageClassification.from_pretrained("Pretrained_Model").to("cuda")

print(model.config.id2label)

{0: 'Real', 1: 'Fake'}


Real과 Fake Label에 맞추어 평가 함수를 수정해주세요!

In [ ]:
# 평가 함수 정의
def evaluate_real_vs_fake(real_images, fake_zip_path, fake_limit=50):
    """
    모델의 Real vs Fake 판별 성능을 평가합니다.
    - 모델의 config.id2label = {0: 'Real', 1: 'Fake'} 기준에 맞춰 처리합니다.
    - 출력: 정확도, 정밀도, 재현율, F1-score (Fake 기준)

    Parameters:
        real_images (list of (str, bytes)): [(파일명, 이미지 bytes)] 형태의 Real 이미지 리스트
        fake_zip_path (str): Fake 이미지가 압축되어 있는 ZIP 파일 경로
        fake_limit (int): 평가에 사용할 최대 Fake 이미지 수
    """

    all_fake_probs = []   # 모델이 예측한 Fake 클래스 (label=1)의 확률
    all_labels = []       # 실제 정답 라벨 (0=Real, 1=Fake)
    all_names = []        # 파일명 저장

    # 1) Real 이미지 예측 (label=0)
    with torch.no_grad():
        for name, img_bytes in real_images[:50]:
            image = Image.open(io.BytesIO(img_bytes)).convert("RGB")
            inputs = processor(images=image, return_tensors="pt").to("cuda")
            outputs = model(**inputs)
            prob_fake = torch.softmax(outputs.logits, dim=1)[0][1].item()  # label=1 (Fake) 확률

            all_fake_probs.append(prob_fake)
            all_labels.append(0)  # Real → label 0
            all_names.append(name)
            print(f"[REAL] {name}: prob(Fake) = {prob_fake:.3f}")

    # 2) Fake 이미지 예측 (label=1)
    with zipfile.ZipFile(fake_zip_path, 'r') as zip_ref:
        fake_images = [n for n in zip_ref.namelist() if n.endswith(('.jpg', '.png'))][:fake_limit]
        with torch.no_grad():
            for name in fake_images:
                image = Image.open(io.BytesIO(zip_ref.read(name))).convert("RGB")
                inputs = processor(images=image, return_tensors="pt").to("cuda")
                outputs = model(**inputs)
                prob_fake = torch.softmax(outputs.logits, dim=1)[0][1].item()  # label=1 (Fake) 확률

                all_fake_probs.append(prob_fake)
                all_labels.append(1)  # Fake → label 1
                all_names.append(name)
                print(f"[FAKE] {name}: prob(Fake) = {prob_fake:.3f}")

    # 3) Best Threshold 탐색 (precision ≈ recall 조건 기준)
    best_thresh = 0.5
    min_diff = float('inf')
    for t in np.arange(0.05, 1.0, 0.01):
        preds = [1 if p > t else 0 for p in all_fake_probs]  # threshold 기준으로 Fake 예측
        prec = precision_score(all_labels, preds, pos_label=1, zero_division=0)
        rec = recall_score(all_labels, preds, pos_label=1, zero_division=0)
        diff = abs(prec - rec)
        if diff < min_diff:
            min_diff = diff
            best_thresh = t

    # 4) 최종 예측 및 성능 출력
    final_preds = [1 if p > best_thresh else 0 for p in all_fake_probs]  # Fake 확률이 크면 Fake로 예측

    print("\n--- 예측 결과 ---")
    for name, prob, label, pred in zip(all_names, all_fake_probs, all_labels, final_preds):
        label_str = "REAL" if label == 0 else "FAKE"
        pred_str = "REAL" if pred == 0 else "FAKE"
        print(f"{name}: GT={label_str}, Pred={pred_str}, prob(Fake)={prob:.3f}")

    acc = accuracy_score(all_labels, final_preds)
    prec = precision_score(all_labels, final_preds, pos_label=1, zero_division=0)
    rec = recall_score(all_labels, final_preds, pos_label=1, zero_division=0)
    f1 = f1_score(all_labels, final_preds, pos_label=1, zero_division=0)

    print(f"\nBest Threshold: {best_thresh:.2f}")
    print(f"Accuracy : {acc:.4f}")
    print(f"Precision: {prec:.4f} (Fake 기준)")
    print(f"Recall   : {rec:.4f} (Fake 기준)")
    print(f"F1-score : {f1:.4f} (Fake 기준)")

In [ ]:
# 평가 실행
evaluate_real_vs_fake(real_images=xx_real, fake_zip_path=xx_fake_path2)

Best Threshold: 0.99
Accuracy : 0.5000
Precision: 0.0000 (Fake 기준)
Recall   : 0.0000 (Fake 기준)
F1-score : 0.0000 (Fake 기준)

# 5. Fine-Tuning

In [ ]:
# FAKE N장 불러오기
fake_data = load_fake_images(xx_fake_path)
dataset = FakeDataset(fake_data, processor)
loader = DataLoader(dataset, batch_size=16, shuffle=True)

optimizer = AdamW(model.parameters(), lr=2e-5)

# 학습 루프 정의
def train_fake(model, loader, epochs=2):
    """
    FAKE 데이터로 모델을 미세조정(Fine-Tuning)합니다.
    """
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch in tqdm(loader, desc=f"Epoch {epoch+1}"):
            inputs = {k: v.to("cuda") for k, v in batch.items()}
            outputs = model(**inputs)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            total_loss += loss.item()
        print(f"Epoch {epoch+1} - Loss: {total_loss / len(loader):.4f}")

# Fine-Tuning 시작
train_fake(model, loader, epochs=2)

Epoch 1: 100%|██████████| 188/188 [00:29<00:00,  6.37it/s]


Epoch 1 - Loss: 0.0269


Epoch 2: 100%|██████████| 188/188 [00:29<00:00,  6.36it/s]

Epoch 2 - Loss: 0.0005


# 6. 모델 저장
save_path 경로 내부에 다음과 같은 파일들이 생성됩니다:
    
    model_fake/
    ├── config.json                # 모델 구조 정보
    ├── pytorch_model.bin         # 모델 가중치 (PyTorch Tensor)
    ├── preprocessor_config.json  # 이미지 전처리 관련 설정
    ├── tokenizer_config.json     # (텍스트 모델용 - Vision 모델은 대부분 없음)
    ├── special_tokens_map.json   # (텍스트 모델용 - Vision 모델은 대부분 없음)

| 파일명                                                  | 설명                                         |
| ---------------------------------------------------- | ------------------------------------------ |
| `pytorch_model.bin`                                  | 학습된 모델의 **가중치 파라미터** 저장 (.bin 형식, PyTorch) |
| `config.json`                                        | 모델의 구조 정의 (예: 레이어 수, hidden size 등)        |
| `preprocessor_config.json`                           | 이미지 전처리 방식 정보 (resize, normalize 등)        |
| `tokenizer_config.json`<br>`special_tokens_map.json` | 주로 텍스트 모델에서 사용. Vision 모델에선 대부분 생성되지 않음    |


In [ ]:
# 학습 완료된 모델과 processor 저장
save_path = "/content/drive/MyDrive/Colab Notebooks/Deepfake_Detection/model_fake"
model.save_pretrained(save_path)
processor.save_pretrained(save_path)

['/content/drive/MyDrive/Colab Notebooks/Deepfake_Detection/vit_ddim_fake/preprocessor_config.json']

# 7. Inference (최종 평가)

In [ ]:
# 저장된 Fine-tuned 모델 불러오기
processor = AutoImageProcessor.from_pretrained(save_path)
model = AutoModelForImageClassification.from_pretrained(save_path).to("cuda").eval()

# 최종 평가
evaluate_real_vs_fake(real_images=xx_real, fake_zip_path=xx_fake_path2)

Best Threshold: 0.99
Accuracy : 0.8600
Precision: 0.7812 (Fake 기준)
Recall   : 1.0000 (Fake 기준)
F1-score : 0.8772 (Fake 기준)

# 결과 해석

**사전학습 모델 (pretrained)** 결과:
- Best Threshold: 0.99
- Accuracy : 0.5000
- Precision: 0.0000 (FAKE 기준)
- Recall   : 0.0000 (FAKE 기준)
- F1-score : 0.0000 (FAKE 기준)

→ 이는 사전학습 모델이 Deepfake (FAKE) 이미지를 전혀 잡아내지 못했다는 의미입니다.

→ threshold를 0.99로 높게 설정해도 확률 예측이 미약하여 실제 FAKE 이미지를 REAL로 오분류한 것입니다.

→ 즉, 실제 fine-tuning된 데이터와 사전학습된 도메인 간 차이가 크기 때문에 성능이 낮게 나옵니다.



---



**Fine-tuning** 모델 결과:
- Best Threshold: 0.99
- Accuracy : 0.8600
- Precision: 0.7812 (FAKE 기준)
- Recall   : 1.0000 (FAKE 기준)
- F1-score : 0.8772 (FAKE 기준)

→ fine-tuning 이후 성능이 크게 향상되었습니다.

→ 특히 recall=1.0은 모든 FAKE 이미지를 빠짐없이 탐지했다는 뜻입니다.

→ precision이 0.7812이므로, FAKE로 예측한 이미지 중 78%가 실제 FAKE였습니다.